# **1. グーグルドライブマウント**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **2. 対象企業の財務諸表取得とcsvファイルへの出力**

In [2]:
import yfinance as yf
import pandas as pd

# 対象のティッカーシンボル
tickers = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'NVDA','TSLA','7203.T']

# データを格納するリスト
financial_data = []

for ticker in tickers:
    # yfinanceを使ってティッカーシンボルのオブジェクトを作成
    company = yf.Ticker(ticker)

    # バランスシート（貸借対照表）
    balance_sheet = company.balance_sheet
    for date in balance_sheet.columns:
        for item in balance_sheet.index:
            value = balance_sheet.loc[item, date]
            if pd.notna(value):  # NaNを除外
                financial_data.append([ticker, date, 'BS', item, value])

    # 損益計算書
    income_statement = company.financials
    for date in income_statement.columns:
        for item in income_statement.index:
            value = income_statement.loc[item, date]
            if pd.notna(value):  # NaNを除外
                financial_data.append([ticker, date, 'PL', item, value])

    # キャッシュフロー計算書
    cash_flow = company.cashflow
    for date in cash_flow.columns:
        for item in cash_flow.index:
            value = cash_flow.loc[item, date]
            if pd.notna(value):  # NaNを除外
                financial_data.append([ticker, date, 'CF', item, value])

# データをデータフレームに変換
df = pd.DataFrame(financial_data, columns=['Ticker', 'Fiscal Date', 'Financial Statement', 'Account Item', 'Value'])

# データフレームをCSVファイルに出力
df.to_csv('/content/drive/My Drive/csv/output/financial_data_all.csv', index=False)

# **3. バランスシート**

In [ ]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 比較する企業のティッカーシンボル
tickers = ['AAPL', 'MSFT', 'AMZN', 'NVDA']

# 必要な項目
required_items = {
    'Current Assets': 'Current Assets',
    'Total Non Current Assets': 'Total Non Current Assets',
    'Total Liabilities Net Minority Interest': 'Total Liabilities Net Minority Interest',
    'Total Equity Gross Minority Interest': 'Total Equity Gross Minority Interest'
}

# 決算期情報を取得してタイトルに設定
titles = []
for ticker in tickers:
    # yfinanceを使ってティッカーシンボルのオブジェクトを作成
    company = yf.Ticker(ticker)

    # バランスシート（貸借対照表）を取得
    balance_sheet = company.balance_sheet.T  # 転置する

    # 最新の決算年月を取得
    latest_date = balance_sheet.index[0]  # 最新の決算期
    latest_date_label = f'{latest_date.year}-{latest_date.month}'

    # タイトルにティッカーと最新の決算期を追加
    titles.append(f'{ticker} ({latest_date_label})')

# サブプロットの作成
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=titles,
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# 各企業のデータを取得して個別のグラフに追加
row, col = 1, 1
for ticker in tickers:
    # yfinanceを使ってティッカーシンボルのオブジェクトを作成
    company = yf.Ticker(ticker)

    # バランスシート（貸借対照表）を取得
    balance_sheet = company.balance_sheet.T  # 転置する

    # 最新の決算年月を取得
    latest_date = balance_sheet.index[0]

    # データを抽出して整理
    data_assets = []
    data_liabilities_equity = []
    for label, item in required_items.items():
        try:
            value = balance_sheet.loc[latest_date, item] / 10**9  # 金額を10億で割る
            if 'Assets' in label:
                data_assets.append(value)
            else:
                data_liabilities_equity.append(value)
        except KeyError:
            if 'Assets' in label:
                data_assets.append(0)  # 項目が見つからない場合は0を設定
            else:
                data_liabilities_equity.append(0)

    # 左側（資産）の積み上げグラフ
    fig.add_trace(go.Bar(
        x=['Assets'],
        y=[data_assets[1]],  # Non-Current Assetsを下に
        name='Total Non Current Assets',
        marker=dict(color='lightgreen'),
        text=[f'Total Non Current Assets: {data_assets[1]:.1f} B'],
        textposition='inside',
        insidetextanchor='middle',
        showlegend=False
    ), row=row, col=col)
    fig.add_trace(go.Bar(
        x=['Assets'],
        y=[data_assets[0]],  # Current Assetsを上に
        name='Current Assets',
        marker=dict(color='lightblue'),
        base=[data_assets[1]],  # Non-Current Assetsの上に積み上げる
        text=[f'Current Assets: {data_assets[0]:.1f} B'],
        textposition='inside',
        insidetextanchor='middle',
        showlegend=False
    ), row=row, col=col)

    # 右側（負債と資本）の積み上げグラフ
    fig.add_trace(go.Bar(
        x=['Liabilities and Equity'],
        y=[data_liabilities_equity[1]],  # Equityを下に
        name='Total Equity',
        marker=dict(color='lightcoral'),
        text=[f'Total Equity: {data_liabilities_equity[1]:.1f} B'],
        textposition='inside',
        insidetextanchor='middle',
        showlegend=False
    ), row=row, col=col)
    fig.add_trace(go.Bar(
        x=['Liabilities and Equity'],
        y=[data_liabilities_equity[0]],  # Liabilitiesを上に
        name='Total Liabilities',
        marker=dict(color='salmon'),
        base=[data_liabilities_equity[1]],  # Equityの上に積み上げる
        text=[f'Total Liabilities: {data_liabilities_equity[0]:.1f} B'],
        textposition='inside',
        insidetextanchor='middle',
        showlegend=False
    ), row=row, col=col)

    # 行と列のインクリメント
    if col == 1:
        col += 1
    else:
        col = 1
        row += 1

# グラフ全体のカスタマイズ
fig.update_layout(
    title='Balance Sheet Comparison of Apple, Microsoft, Amazon, NVIDIA - Latest Period',
    height=900,
    showlegend=True,
    barmode='stack'  # 純粋な積み上げにするための設定
)

# 共通の凡例を追加
fig.add_trace(go.Bar(name='Total Non Current Assets', marker=dict(color='lightgreen')), row=1, col=1)
fig.add_trace(go.Bar(name='Current Assets', marker=dict(color='lightblue')), row=1, col=1)
fig.add_trace(go.Bar(name='Total Equity', marker=dict(color='lightcoral')), row=1, col=1)
fig.add_trace(go.Bar(name='Total Liabilities', marker=dict(color='salmon')), row=1, col=1)

# グラフの表示
fig.show()

# **4. 損益計算書**

In [ ]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 比較する企業のティッカーシンボル
tickers = ['AAPL', 'MSFT', 'AMZN', 'NVDA']

# 必要な項目
required_items = {
    'Total Revenue': 'Total Revenue',
    'Gross Profit': 'Gross Profit',
    'Operating Income': 'Operating Income',
    'Pretax Income': 'Pretax Income',
    'Net Income': 'Net Income'
}

# 決算期情報を取得してタイトルに設定
titles = []
data = {ticker: {} for ticker in tickers}  # 各企業のデータを格納する辞書

for ticker in tickers:
    # yfinanceを使ってティッカーシンボルのオブジェクトを作成
    company = yf.Ticker(ticker)

    # 損益計算書（PL）を取得
    income_statement = company.financials.T

    # 最新の決算年月を取得
    latest_date = income_statement.index[0]
    latest_date_label = f'{latest_date.year}-{latest_date.month}'

    # タイトルにティッカーと最新の決算期を追加
    titles.append(f'{ticker} ({latest_date_label})')

    # データを抽出して整理
    for label, item in required_items.items():
        try:
            data[ticker][label] = income_statement.loc[latest_date, item] / 10**9  # 金額を10億で割る
        except KeyError:
            data[ticker][label] = 0  # 項目が見つからない場合は0を設定

# サブプロットの作成
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=titles,
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# 各企業のデータを取得して個別のグラフに追加
row, col = 1, 1
for ticker in tickers:
    fig.add_trace(go.Bar(
        name=ticker,
        x=list(required_items.keys()),
        y=[data[ticker]['Total Revenue'],
           data[ticker]['Gross Profit'],
           data[ticker]['Operating Income'],
           data[ticker]['Pretax Income'],
           data[ticker]['Net Income']],
        text=[f'{data[ticker]["Total Revenue"]:.2f} B',
              f'{data[ticker]["Gross Profit"]:.2f} B',
              f'{data[ticker]["Operating Income"]:.2f} B',
              f'{data[ticker]["Pretax Income"]:.2f} B',
              f'{data[ticker]["Net Income"]:.2f} B'],
        textposition='auto'
    ), row=row, col=col)

    # 行と列のインクリメント
    if col == 1:
        col += 1
    else:
        col = 1
        row += 1

# グラフ全体のカスタマイズ
fig.update_layout(
    title='Profit and Loss Statement Comparison of AAPL, Microsoft, Amazon, NVIDIA - Latest Period',
    height=900,
    showlegend=False,
    barmode='group'
)

# グラフの表示
fig.show()

# **5. トヨタの過去3年のROE、ROIC計算**

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# トヨタ自動車の株式データを取得
ticker = "7203.T"  # トヨタ自動車のティッカーシンボル
stock = yf.Ticker(ticker)

# 財務データを取得
financials = stock.financials.T
balance_sheet = stock.balance_sheet.T

# データの準備
net_income = financials["Net Income"].head(3)  # 過去3年分の純利益
common_stock_equity = balance_sheet["Common Stock Equity"].head(3)  # 過去3年分の普通株持分
operating_income = financials["Operating Income"].head(3)  # 過去3年分の営業利益
tax_rate = 0.30  # トヨタの仮の実効税率（最新の実効税率を使用するのが望ましい）

# ROEの計算
roe = (net_income / common_stock_equity) * 100

# NOPAT（税後営業利益）の計算
nopat = operating_income * (1 - tax_rate)

# 投下資本サイドからの計算
invested_capital = balance_sheet["Invested Capital"].head(3)  # 過去3年分の投下資本
cash_and_cash_equivalents = balance_sheet["Cash And Cash Equivalents"].head(3)  # 過去3年分の現金および現金同等物
# 現金および現金同等物を差し引いた投下資本
net_invested_capital = invested_capital - cash_and_cash_equivalents

# 投下資本サイドからのROICの計算
roic = (nopat / net_invested_capital) * 100

# 資金運用サイドからのROICの計算
fixed_assets = balance_sheet["Total Non Current Assets"].head(3)  # 過去3年分の固定資産
current_assets = balance_sheet["Current Assets"].head(3)  # 過去3年分の流動資産
current_liabilities = balance_sheet["Current Liabilities"].head(3)  # 過去3年分の流動負債
# 運転資金の計算
working_capital = current_assets - current_liabilities
# 資金運用サイドの投下資本
operating_assets = fixed_assets + working_capital - cash_and_cash_equivalents

# 資金運用サイドからのROICの計算
roic_asset_side = (nopat / operating_assets) * 100

# 年度の情報
years = roe.index.year

# 年が新しくなる順に並べ替え
years = years[::-1]
roe = roe[::-1]
roic = roic[::-1]
roic_asset_side = roic_asset_side[::-1]

# データフレームを作成
roe_roic_df = pd.DataFrame({
    'Year': years,
    'ROE (%)': roe.values,
    'ROIC Liability Side (%)': roic.values,
    'ROIC Asset Side (%)': roic_asset_side.values
}).set_index('Year')

# データフレームの表示
print(roe_roic_df)

# 最大値を計算（全てのグラフで共有するため）
max_y = max(roe.max(), roic.max(), roic_asset_side.max()) + 2  # 少し余裕を持たせるため+2

# グラフの作成
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 6))

# ROEの棒グラフ
axes[0].bar(years, roe, color='b', width=0.5)
axes[0].set_title('Toyota ROE (2022-2024)')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('ROE (%)')
axes[0].set_ylim(0, max_y)  # 縦軸の最大値を合わせる
axes[0].grid(True)
axes[0].set_xticks(years)  # X軸の目盛りを整数に設定

# 投下資本サイドからのROICの棒グラフ
axes[1].bar(years, roic, color='r', width=0.5)
axes[1].set_title('Toyota ROIC Liability Side (2022-2024)')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('ROIC (%)')
axes[1].set_ylim(0, max_y)  # 縦軸の最大値を合わせる
axes[1].grid(True)
axes[1].set_xticks(years)  # X軸の目盛りを整数に設定

# 資金運用サイドからのROICの棒グラフ
axes[2].bar(years, roic_asset_side, color='g', width=0.5)
axes[2].set_title('Toyota ROIC Asset Side (2022-2024)')
axes[2].set_xlabel('Year')
axes[2].set_ylabel('ROIC (%)')
axes[2].set_ylim(0, max_y)  # 縦軸の最大値を合わせる
axes[2].grid(True)
axes[2].set_xticks(years)  # X軸の目盛りを整数に設定

# グラフの調整
plt.tight_layout()

# グラフの表示
plt.show()

# **6. トヨタとテスラのROEツリー**

In [21]:
import yfinance as yf
import pandas as pd

# トヨタ自動車とテスラのティッカーシンボル
tickers = {"Toyota": "7203.T", "Tesla": "TSLA"}

# 財務データを取得し、最新の年のデータを抽出する関数
def get_latest_annual_data(ticker):
    stock = yf.Ticker(ticker)
    PL = stock.financials.T / 10**6  # 百万単位に変換
    BS = stock.balance_sheet.T / 10**6  # 百万単位に変換

    # インデックスを年ベースに変換
    PL.index = pd.to_datetime(PL.index).year
    BS.index = pd.to_datetime(BS.index).year

    # 最新の年のデータを抽出
    latest_year = PL.index.max()
    return PL.loc[latest_year], BS.loc[latest_year]

# 各企業のデータを取得
financial_data = {company: get_latest_annual_data(ticker) for company, ticker in tickers.items()}

# 各指標を計算する関数
def calculate_ratios(PL, BS):
    # 基本項目の取得
    revenue = PL["Total Revenue"]
    cost_of_revenue = PL.get("Cost Of Revenue", 0)
    sga_expense = PL.get("Selling General And Administration", 0)
    operating_income = PL["Operating Income"]
    net_income = PL["Net Income"]

    current_assets = BS["Current Assets"]
    fixed_assets = BS["Total Non Current Assets"]

    cash_and_cash_equivalents = BS["Cash And Cash Equivalents"]
    accounts_receivable = BS["Accounts Receivable"]
    inventory = BS.get("Inventory")
    current_liabilities = BS["Current Liabilities"]
    accounts_payable = BS["Accounts Payable"]
    total_equity = BS["Common Stock Equity"]
    total_assets = BS["Total Assets"]
    invested_capital = BS["Invested Capital"]


    # 有利子負債の定義
    total_debt = BS.get("Total Debt", 0)
    capital_lease_obligations = BS.get("Capital Lease Obligations", 0)
    interest_bearing_debt = total_debt - capital_lease_obligations

    # 財務指標の計算
    nopat = operating_income * (1 - 0.30)  # 税後営業利益（仮定として30%の税率）
#    invested_capital = total_assets - current_liabilities
    net_invested_capital = invested_capital - cash_and_cash_equivalents

    # 運転資金の計算
    working_capital = current_assets - current_liabilities
    # 資金運用サイドの投下資本
    operating_assets = fixed_assets + working_capital - cash_and_cash_equivalents

    roe = (net_income / total_equity)
    roic = (nopat / operating_assets)
    financial_leverage = operating_assets / total_equity

    roic_liab = nopat / net_invested_capital
    equity_multiplier = net_invested_capital / total_equity
    operating_profit_margin = operating_income / revenue
    sga_ratio = sga_expense / revenue
    cog_ratio = cost_of_revenue / revenue

    asset_turnover = revenue / operating_assets
    fixed_asset_turnover = revenue / fixed_assets

    # 回転期間（月単位）
    working_capital_turnover_period = (current_assets - current_liabilities) / (revenue / 12)
    ar_turnover_period = accounts_receivable / (revenue / 12)
    inventory_turnover_period = inventory / (revenue / 12) if inventory is not None else None
    ap_turnover_period = accounts_payable / (revenue / 12) if accounts_payable is not None else None

    return {
        "Revenue": revenue,
        "Cost of Revenue": cost_of_revenue,
        "SGA Expense": sga_expense,
        "Operating Income": operating_income,
        "Net Income": net_income,
        "Current Assets": current_assets,
        "Cash and Cash Equivalents": cash_and_cash_equivalents,
        "Accounts Receivable": accounts_receivable,
        "Inventory": inventory,
        "Current Liabilities": current_liabilities,
        "Accounts Payable": accounts_payable,
        "Total Equity": total_equity,
        "Interest Bearing Debt": interest_bearing_debt,
        "Total Assets": total_assets,
        "ROE": roe,
        "ROIC assets": roic,
        "Financial Leverage": financial_leverage,
        "ROIC liabilities": roic_liab,
        "Equity Multiplier": equity_multiplier,
        "Operating Profit Margin": operating_profit_margin,
        "COG Expense Ratio": cog_ratio,
        "SGA Expense Ratio": sga_ratio,
        "Asset Turnover": asset_turnover,
        "Fixed Asset Turnover": fixed_asset_turnover,
        "Working Capital Turnover Period (months)": working_capital_turnover_period,
        "Accounts Receivable Turnover Period (months)": ar_turnover_period,
        "Inventory Turnover Period (months)": inventory_turnover_period,
        "Accounts Payable Turnover Period (months)": ap_turnover_period
    }

# 各企業の指標を計算
ratios = {company: calculate_ratios(PL, BS) for company, (PL, BS) in financial_data.items()}

# 基本項目と計算した指標をデータフレームにまとめる
ratios_df = pd.DataFrame(ratios).T

# データフレームの表示
#print(ratios_df)

# データをCSVファイルに出力
ratios_df.to_csv('/content/drive/My Drive/csv/output/financial_ratios_with_items.csv', encoding='utf-8-sig')